In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

In [ ]:
def lstsqk(x, y):
    return np.multiply(x, y).mean(axis = 0) / np.multiply(x, x).mean(axis = 0)

In [ ]:
def lstsqkerr(x, y):
    return 1 / np.sqrt(len(x)) * np.sqrt(np.multiply(y, y).mean(axis = 0) / np.multiply(x, x).mean(axis = 0) - lstsqk(x,y) ** 2)

# First

In [ ]:
df1 = pd.read_excel('experimental_data.xlsx', sheet_name = 'Data').reset_index(drop = True)

In [ ]:
rhogh = 88 # Pa
g = 9.81
rho = 1000
r = 1.05e-3 / 2
tzero = -273.15 # Kelvin
sigma_h = 1
sina = 0.2
sigma_r = 0.025e-3 / 2 # mm

In [ ]:
tlist = list(df1['T, C'])

df1['H_mean, мм'] = df1['T, C']
df1['H_sigma, мм'] = df1['T, C']
df1['dP, Па'] = df1['T, C']
df1['k'] = df1['T, C']

df1['T, K'] = df1['T, C'] - tzero
for t in tlist:
    s = df1.loc[df1['T, C'] == t].iloc[0]['H1':'H6']
    df1.loc[df1['T, C'] == t, 'H_mean, мм'] = s.mean() * sina
    df1.loc[df1['T, C'] == t, 'H_sigma, мм'] = np.sqrt(s.std() ** 2 + sigma_h ** 2) * sina
    df1.loc[df1['T, C'] == t, 'dP, Па'] = df1.loc[df1['T, C'] == t, 'H_mean, мм'] * rho * g / 1e3 - rhogh
    df1.loc[df1['T, C'] == t, 'k'] = df1.loc[df1['T, C'] == t, 'dP, Па'] * r / 2

df1['sigma_dP, Па'] = df1['H_sigma, мм'] * rho * g / 1e3
df1['sigma_k'] = df1['k'] * np.sqrt((df1['sigma_dP, Па'] / df1['dP, Па']) ** 2 + (sigma_r / r) ** 2)

In [ ]:
df1

# General Data

In [ ]:
p1, v1 = np.polyfit(df1['T, K'], df1['k'], deg = 1, cov = True)
err1 = np.sqrt(np.diag(v1))

In [ ]:
def calculate_least_squares():
    df = df1[:].copy()
    
    df['T * k'] = df['T, K'] * df['k']
    
    df['T^2, K^2'] = df['T, K'] * df['T, K']
    df['k^2'] = df['k'] * df['k']
    
    x_m = df['T, K'].mean()
    y_m = df['k'].mean()
    
    xx_m = df['T^2, K^2'].mean()
    xy_m = df['T * k'].mean()
    yy_m = df['k^2'].mean()
    
    return df, x_m, y_m, xx_m, xy_m, yy_m

In [ ]:
mnkdata = []
df, _, _, _, _, _ = calculate_least_squares()
mnkdata.append(df)

In [ ]:
for i in range(len(mnkdata)):
    mnkdata[i].to_excel('plots/output{}.xlsx'.format(i + 1))

# Plot Linear Scale

In [ ]:
def get_min_max(series, k1 = 3 / 4, k2 = 11 / 10):
    return k1 * min(series), k2 * max(series)

In [ ]:
def plot_data_linear_scale(ax, df, p, e, yerr):
    y_min, y_max = get_min_max(df['k'], k1 = 9 / 10, k2 = 11 / 10)
    x_min, x_max = get_min_max(df['T, K'], k1 = 9 / 10, k2 = 11 / 10)
    
    ax.xaxis.set_major_locator(plt.MultipleLocator(10))
    ax.yaxis.set_major_locator(plt.MultipleLocator(1e-3))
    ax.xaxis.set_minor_locator(plt.MultipleLocator(5))
    ax.yaxis.set_minor_locator(plt.MultipleLocator(1e-3))
    
    plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
    
    ax.set_ylim(bottom = y_min, top = y_max)
    ax.set_xlim(left = x_min, right = x_max)
    
    ax.grid(visible = True, linestyle = '--', which = 'both')
    x = np.linspace(x_min, x_max, 1000)
    ax.errorbar(df['T, K'], df['k'], fmt = 'o')
    ax.plot(x, p[0] * x + p[1], label = "Линейная модель: $\sigma(T) = a + b \cdot T$")
    ax.plot([], [], ' ', label = '$b = {k:.3e} \pm {dk:.3e}, a = {C:.3e} \pm {dC:.3e}$'.format(k = p[0], dk = e[0], C = p[1], dC = e[1]))
    
    ax.legend()
    ax.set_xlabel('$T, K$')
    ax.set_ylabel('$\sigma(T), Н/м$')

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize=(11.7, 8.3))
fig.suptitle('Рис. 1. Зависимость $\sigma(T)$ от температуры $T$')
plot_data_linear_scale(ax, df1, p1, err1, df1['sigma_k'])
fig.savefig('plots/plot1.pdf')

In [ ]:
df1['q, Н/м'] = - df1['T, K'] * p1[0]
p2, v2 = np.polyfit(df1['T, K'], df1['q, Н/м'], deg = 1, cov = True)
err2 = np.sqrt(np.diag(v2))

In [ ]:
def plot_data_heat(ax, df, p, e, yerr):
    y_min, y_max = get_min_max(df['q, Н/м'], k1 = 9 / 10, k2 = 11 / 10)
    x_min, x_max = get_min_max(df['T, K'], k1 = 9 / 10, k2 = 11 / 10)
    
    ax.xaxis.set_major_locator(plt.MultipleLocator(10))
    ax.yaxis.set_major_locator(plt.MultipleLocator(1e-3))
    ax.xaxis.set_minor_locator(plt.MultipleLocator(5))
    ax.yaxis.set_minor_locator(plt.MultipleLocator(1e-3))
    
    plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
    
    ax.set_ylim(bottom = y_min, top = y_max)
    ax.set_xlim(left = x_min, right = x_max)
    
    ax.grid(visible = True, linestyle = '--', which = 'both')
    x = np.linspace(x_min, x_max, 1000)
    ax.errorbar(df['T, K'], df['q, Н/м'], fmt = 'o')
    ax.plot(x, p[0] * x + p[1])
    
    ax.set_xlabel('$T, K$')
    ax.set_ylabel('$q, Н/м$')

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize=(11.7, 8.3))
fig.suptitle('Рис. 2. Зависимость $q(T)$ от температуры $T$')
plot_data_heat(ax, df1, p2, err2, df1['sigma_k'])
fig.savefig('plots/plot2.pdf')

In [ ]:
df1['U, (Н/м)^2'] = - df1['k'] * df1['T, K'] * p1[0]
p3, v3 = np.polyfit(df1['T, K'], df1['q, Н/м'], deg = 1, cov = True)
err3 = np.sqrt(np.diag(v3))

In [ ]:
def plot_data_enery(ax, df):
    y_min, y_max = get_min_max(df['U, (Н/м)^2'], k1 = 95 / 100, k2 = 105 / 100)
    x_min, x_max = get_min_max(df['T, K'], k1 = 95 / 100, k2 = 105 / 100)
    
    ax.xaxis.set_major_locator(plt.MultipleLocator(5))
    ax.yaxis.set_major_locator(plt.MultipleLocator(5e-5))
    ax.xaxis.set_minor_locator(plt.MultipleLocator(2.5))
    ax.yaxis.set_minor_locator(plt.MultipleLocator(2.5e-5))
    
    plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
    
    ax.set_ylim(bottom = y_min, top = y_max)
    ax.set_xlim(left = x_min, right = x_max)
    
    ax.grid(visible = True, linestyle = '--', which = 'both')
    x = np.linspace(x_min, x_max, 1000)
    ax.errorbar(df['T, K'], df['U, (Н/м)^2'], marker = 'd')
    
    ax.set_xlabel('$T, K$')
    ax.set_ylabel('$U/F, (Н/м)^2$')

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize=(11.7, 8.3))
fig.suptitle('Рис. 3. Зависимость $U/F(T)$ от температуры $T$')
plot_data_enery(ax, df1)
fig.savefig('plots/plot3.pdf')